In [1]:
%load_ext autoreload
%autoreload 2
import warnings
from IPython.core.interactiveshell import InteractiveShell

warnings.filterwarnings('ignore')
InteractiveShell.ast_node_interactivity = 'last_expr'

In [2]:
import configparser

config = configparser.ConfigParser()
config.read('config.ini')

['config.ini']

In [3]:
import flash
from flash.core.data.utils import download_data

download_data(config['text-example']['download_url'], 'data/')

data/imdb.zip: 15576KB [00:02, 7100.61KB/s]                           


In [4]:
from flash.text import TextClassificationData, TextClassifier

input_fields,target_fields = 'review', 'sentiment'
batch_size=512
datamodule = TextClassificationData.from_csv(
    train_file=config['text-example']['train_file'],
    val_file=config['text-example']['val_file'],
    test_file=config['text-example']['test_file'],
    input_fields=input_fields,
    target_fields=target_fields,
    batch_size=batch_size
)

Using custom data configuration default
100%|██████████| 23/23 [00:20<00:00,  1.13ba/s]
Using custom data configuration default
100%|██████████| 3/3 [00:02<00:00,  1.24ba/s]
Using custom data configuration default
100%|██████████| 3/3 [00:02<00:00,  1.21ba/s]


In [7]:
model = TextClassifier(num_classes=datamodule.num_classes, backbone='prajjwal1/bert-mini')

Downloading: 100%|██████████| 286/286 [00:00<00:00, 26.0kB/s]
Downloading: 100%|██████████| 45.1M/45.1M [00:04<00:00, 10.3MB/s]
Some weights of the model checkpoint at prajjwal1/bert-mini were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenc

**Backbone** Can be seen on huggingface: [huggingface-link](https://huggingface.co/prajjwal1/bert-mini)